In [1]:
import openmeteo_requests, requests_cache
import pandas as pd
from retry_requests import retry

In [8]:
# parameters: lat, lon, variables to include in fetched data, date range (June 2023 - June 2025)
params_dhaka = {
	"latitude": 23.76,
	"longitude": 90.38,
	"city": "dhaka",
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "uv_index_clear_sky", "uv_index", "dust", "aerosol_optical_depth", "us_aqi"],
	"domains": "cams_global",
	"start_date": "2025-06-30",
	"end_date": "2025-08-20"
}

params_chittagong = {
	"latitude": 22.33,
	"longitude": 91.83,
	"city": "chittagong",
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "uv_index_clear_sky", "uv_index", "dust", "aerosol_optical_depth", "us_aqi"],
	"domains": "cams_global",
	"start_date": "2025-06-30",
	"end_date": "2025-08-20"
}

params_khulna = {
	"latitude": 22.81,
	"longitude": 89.55,
	"city": "khulna",
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "uv_index_clear_sky", "uv_index", "dust", "aerosol_optical_depth", "us_aqi"],
	"domains": "cams_global",
	"start_date": "2025-06-30",
	"end_date": "2025-08-20"
}

In [9]:
param_list = [params_dhaka, params_chittagong, params_khulna]

In [10]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://air-quality-api.open-meteo.com/v1/air-quality"

In [11]:
dataframes = []

In [12]:
for params in param_list:
    city = params.pop("city")
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
    hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
    hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
    hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
    hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
    hourly_ozone = hourly.Variables(5).ValuesAsNumpy()
    hourly_uv_index_clear_sky = hourly.Variables(6).ValuesAsNumpy()
    hourly_uv_index = hourly.Variables(7).ValuesAsNumpy()
    hourly_dust = hourly.Variables(8).ValuesAsNumpy()
    hourly_aerosol_optical_depth = hourly.Variables(9).ValuesAsNumpy()
    hourly_us_aqi = hourly.Variables(10).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["city"] = city

    hourly_data["pm10"] = hourly_pm10
    hourly_data["pm2_5"] = hourly_pm2_5
    hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
    hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
    hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
    hourly_data["ozone"] = hourly_ozone
    hourly_data["uv_index_clear_sky"] = hourly_uv_index_clear_sky
    hourly_data["uv_index"] = hourly_uv_index
    hourly_data["dust"] = hourly_dust
    hourly_data["aerosol_optical_depth"] = hourly_aerosol_optical_depth
    hourly_data["us_aqi"] = hourly_us_aqi

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    dataframes.append(hourly_dataframe)

Coordinates 23.800003051757812°N 90.39999389648438°E
Elevation 13.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
Coordinates 22.300003051757812°N 91.80001831054688°E
Elevation 10.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
Coordinates 22.800003051757812°N 89.5°E
Elevation 7.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


In [13]:
final_df = pd.concat(dataframes, axis=0)
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3744 entries, 0 to 1247
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   date                   3744 non-null   datetime64[ns, UTC]
 1   city                   3744 non-null   object             
 2   pm10                   3744 non-null   float32            
 3   pm2_5                  3744 non-null   float32            
 4   carbon_monoxide        3744 non-null   float32            
 5   nitrogen_dioxide       3744 non-null   float32            
 6   sulphur_dioxide        3744 non-null   float32            
 7   ozone                  3744 non-null   float32            
 8   uv_index_clear_sky     3744 non-null   float32            
 9   uv_index               3744 non-null   float32            
 10  dust                   3744 non-null   float32            
 11  aerosol_optical_depth  3744 non-null   float32            
 1

In [14]:
final_df.to_csv("dhaka_ctg_khulna_aqi_data_recent.csv", index=False)